In [15]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import medfilt
import numpy as np
from datasets import *
from models import *

from torch.utils.tensorboard import SummaryWriter

In [16]:
root_dir = "../csvs/collected_data"
batch_size = 256
transforms = transforms.Compose([
        DataframeToNumpy(),
        NormalizeAndFilter(median_filter=False),
        ToTensor(),
    ])
    
dataset = Gestures(root_dir, None, train=True)
train_subset, val_subset = torch.utils.data.random_split(dataset, [int(len(dataset) * 0.9), len(dataset) - int(len(dataset) * .9)])
train_set = SubsetWrapper(train_subset, transform=transforms)
val_set = SubsetWrapper(val_subset, transform=transforms)
test_set = Gestures(root_dir, transforms, train=False, test=True)

# create the data loaders
train_loader = DataLoader(train_set,batch_size=batch_size,shuffle=True,collate_fn=varying_length_collate_fn)
val_loader = DataLoader(val_set,batch_size=batch_size,collate_fn=varying_length_collate_fn)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,collate_fn=varying_length_collate_fn)

In [18]:
model = RNNModel(63,256,1,25,device='cpu').to('cpu')
# model_params = dict(input_dim=63, num_classes=25, num_heads=6, hidden_dim=276, num_layers=1)
# model = TransformerClassifier(**model_params)#.to('cuda')
model.load_state_dict(torch.load("models/collected.pth")['model_state_dict'])
model.eval()

activation = {}
activation['i'] = 0
activation['emb'] = torch.zeros((125,256))
def get_activation(name):
    def hook(model, input, output):
        activation['i'] += 1
        activation[name] += input[0].detach()
    return hook

model.fc.register_forward_hook(get_activation('emb'))
# model.transformer_feature_extractor.register_forward_hook(get_activation('emb'))

In [29]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
# init tensorboard
writer = SummaryWriter()

with torch.no_grad():
    batch_imgs, batch_labels, _ = next(iter(test_loader))
    preds = model(batch_imgs.to('cpu'))
    embds = activation['emb'].squeeze(1).to('cpu')
    embds /= activation['i']

    # get the labels
    batch_label_strings = []
    for idx,label in enumerate(batch_labels):
        batch_label_strings.append(str(label.item()))

    writer.add_embedding(embds,metadata=batch_label_strings)#,label_img=batch_imgs)

transformer

In [ ]:
import tensorflow as tf
import tensorboard as tb
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile
# init tensorboard
writer = SummaryWriter()

embds = torch.zeros((batch_size,276))
with torch.no_grad():
    batch_imgs, batch_labels, _ = next(iter(test_loader))
    for i,img in enumerate(batch_imgs):
        preds = model(img.unsqueeze(0))
        embds[i,:] = activation['emb']

    # get the labels
    batch_label_strings = []
    for idx,label in enumerate(batch_labels):
        batch_label_strings.append(str(label.item()))

    writer.add_embedding(embds,metadata=batch_label_strings)#,label_img=batch_imgs)